In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "cleaned_unsw_output.csv"
test_data = pd.read_csv(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2151027
1      91904
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
0    92581
1    25848
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'DoS', 'Fuzzers', 'Reconnaissance'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
# test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,123,123,123,123,123,123,123,123,123,123,...,123,123,123,123,123,123,123,123,123,123
Backdoor,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
Benign,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103,...,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103
DoS,505,505,505,505,505,505,505,505,505,505,...,505,505,505,505,505,505,505,505,505,505
Exploits,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882,...,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882
Fuzzers,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559,...,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559
Generic,476,476,476,476,476,476,476,476,476,476,...,476,476,476,476,476,476,476,476,476,476
Reconnaissance,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
Shellcode,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159


In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,92581,92581,92581,92581,92581,92581,92581,92581,92581,92581,...,92581,92581,92581,92581,92581,92581,92581,92581,92581,92581
DoS,11940,11940,11940,11940,11940,11940,11940,11940,11940,11940,...,11940,11940,11940,11940,11940,11940,11940,11940,11940,11940
Fuzzers,7796,7796,7796,7796,7796,7796,7796,7796,7796,7796,...,7796,7796,7796,7796,7796,7796,7796,7796,7796,7796
Reconnaissance,6112,6112,6112,6112,6112,6112,6112,6112,6112,6112,...,6112,6112,6112,6112,6112,6112,6112,6112,6112,6112


In [16]:
# X = data.drop(columns=["Attack", "Label"])
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X_train, y_train = X, y
X_test = test_data.drop(columns=["Attack", "Label"])
y_test = test_data[["Attack", "Label"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape
##

((224295, 41), (118429, 41), (224295, 2), (118429, 2))

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
28228,175.45.176.3,149.171.126.19,0.000002,0.000004,0.028485,0.000341,0.045794,0.000341,5.420943e-07,5.797048e-07,...,0.558214,0.558214,0.000007,0.000007,0.000002,0.000002,0.0,0.000002,"[1.7197633617740213e-06, 4.298851310470571e-06...",28228
856405,175.45.176.1,149.171.126.11,0.000001,0.000003,0.022255,0.000273,0.035816,0.000219,4.349885e-07,4.651680e-07,...,0.447924,0.447924,0.000009,0.000009,0.000001,0.000001,0.0,0.000001,"[1.3799764291286633e-06, 3.449494048215318e-06...",856405
465045,175.45.176.1,149.171.126.11,0.000002,0.000005,0.031897,0.000394,0.043554,0.000315,6.265204e-07,6.699884e-07,...,0.645151,0.645151,0.000005,0.000005,0.000002,0.000002,0.0,0.000002,"[1.9876005740782747e-06, 4.968357579006962e-06...",465045
959848,175.45.176.3,149.171.126.10,0.000002,0.000005,0.029782,0.000370,0.047666,0.000370,5.878823e-07,6.286695e-07,...,0.605364,0.605364,0.000013,0.000013,0.000002,0.000002,0.0,0.000002,"[1.8650231844538396e-06, 4.661953812225061e-06...",959848
1381572,175.45.176.0,149.171.126.14,0.000002,0.000004,0.028129,0.000351,0.038160,0.000281,5.580212e-07,5.967367e-07,...,0.574615,0.574615,0.000016,0.000016,0.000002,0.000002,0.0,0.000002,"[1.7702906951465035e-06, 4.425153276258943e-06...",1381572


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_UNSW_v3.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [32]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v3.pkl'))

<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [38]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [39]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [40]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.168109,-0.144782,0.125198,0.026733,-0.046728,0.034635,0.235400,0.162897,0.250891,0.004992,...,0.0,9.119847e-07,0.000976,0.000990,1.443004e-10,1.442974e-10,7.127674e-10,7.128415e-10,0.0,6.870455e-10
1,0.168109,-0.144782,0.125198,0.026733,-0.046728,0.034635,0.235400,0.162897,0.250891,0.004992,...,0.0,4.974424e-07,0.000533,0.000540,7.870870e-11,7.870709e-11,3.887792e-10,3.888196e-10,0.0,3.747492e-10
2,0.168109,-0.144782,0.125198,0.026733,-0.046728,0.034635,0.235400,0.162897,0.250891,0.004992,...,0.0,5.803429e-07,0.000621,0.000630,9.182578e-11,9.182390e-11,4.535706e-10,4.536178e-10,0.0,4.372025e-10
3,0.168109,-0.144782,0.125198,0.026733,-0.046728,0.034635,0.235400,0.162897,0.250891,0.004992,...,0.0,4.971614e-07,0.000532,0.000540,7.866425e-11,7.866264e-11,3.885596e-10,3.886000e-10,0.0,3.745375e-10
4,0.168109,-0.144782,0.125198,0.026733,-0.046728,0.034635,0.235400,0.162897,0.250891,0.004992,...,0.0,5.803549e-07,0.000621,0.000630,9.182768e-11,9.182580e-11,4.535800e-10,4.536272e-10,0.0,4.372115e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236853,0.169167,-0.190891,0.135045,0.054692,-0.081589,0.038151,0.254963,0.178242,0.202676,-0.005593,...,0.0,0.000000e+00,0.000000,0.002153,1.995649e-08,1.995608e-08,9.857445e-08,9.858470e-08,0.0,9.501716e-08
236854,0.169167,-0.190891,0.135045,0.054692,-0.081589,0.038151,0.254963,0.178242,0.202676,-0.005593,...,0.0,0.000000e+00,0.000000,0.002153,1.995649e-08,1.995608e-08,9.857445e-08,9.858470e-08,0.0,9.501716e-08
236855,0.169167,-0.190891,0.135045,0.054692,-0.081589,0.038151,0.254963,0.178242,0.202676,-0.005593,...,0.0,0.000000e+00,0.000000,0.002153,1.995649e-08,1.995608e-08,9.857445e-08,9.858470e-08,0.0,9.501716e-08
236856,0.169167,-0.190891,0.135045,0.054692,-0.081589,0.038151,0.254963,0.178242,0.202676,-0.005593,...,0.0,0.000000e+00,0.000000,0.002153,1.995649e-08,1.995608e-08,9.857445e-08,9.858470e-08,0.0,9.501716e-08


In [41]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [42]:
from pyod.models.cblof import CBLOF
n_est = [5,6,7,9,10] # can't be lower than 5, as it will not work with the dataset
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_fuse_train_samples)
    except Exception as e:
        print(f"Error with n_estimators={n_est} and contamination={con}: {e}")
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [03:46<00:00,  7.55s/it]

{'n_estimators': 7, 'con': 0.001}
0.43851823453340866
              precision    recall  f1-score   support

           0     0.7816    0.9989    0.8770    185162
           1     0.0096    0.0000    0.0001     51696

    accuracy                         0.7809    236858
   macro avg     0.3956    0.4995    0.4385    236858
weighted avg     0.6131    0.7809    0.6856    236858



In [43]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [03:50<00:00,  7.69s/it]

{'n_estimators': 5, 'con': 0.04}
0.5832144786276788
              precision    recall  f1-score   support

           0     0.8101    0.9429    0.8715    185162
           1     0.5048    0.2083    0.2949     51696

    accuracy                         0.7826    236858
   macro avg     0.6574    0.5756    0.5832    236858
weighted avg     0.7435    0.7826    0.7457    236858



In [44]:
# HBOS  Embeddings+Raw (Multimodal)

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [46]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:59<00:00,  4.98s/it]

{'n_estimators': 5, 'con': 0.001}
0.17915537473055304
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    185162
           1     0.2183    1.0000    0.3583     51696

    accuracy                         0.2183    236858
   macro avg     0.1091    0.5000    0.1792    236858
weighted avg     0.0476    0.2183    0.0782    236858




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [47]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [03:07<00:00,  5.21s/it]

{'n_estimators': 5, 'con': 0.05}
0.17915537473055304
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    185162
           1     0.2183    1.0000    0.3583     51696

    accuracy                         0.2183    236858
   macro avg     0.1091    0.5000    0.1792    236858
weighted avg     0.0476    0.2183    0.0782    236858




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [48]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [49]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:32<00:00,  4.23s/it]

{'n_estimators': 5, 'con': 0.001}
0.17915537473055304
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    185162
           1     0.2183    1.0000    0.3583     51696

    accuracy                         0.2183    236858
   macro avg     0.1091    0.5000    0.1792    236858
weighted avg     0.0476    0.2183    0.0782    236858




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [50]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:44<00:00,  4.57s/it]

{'n_estimators': 5, 'con': 0.05}
0.17915537473055304
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    185162
           1     0.2183    1.0000    0.3583     51696

    accuracy                         0.2183    236858
   macro avg     0.1091    0.5000    0.1792    236858
weighted avg     0.0476    0.2183    0.0782    236858




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [51]:
##  IF  Emb+Raw

In [52]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [53]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:01<00:31,  1.37s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:02<00:26,  1.20s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:03<00:25,  1.23s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%

{'n_estimators': 20, 'con': 0.001}
0.4521751521778899
              precision    recall  f1-score   support

           0     0.7817    0.9814    0.8702    185162
           1     0.2171    0.0185    0.0341     51696

    accuracy                         0.7712    236858
   macro avg     0.4994    0.4999    0.4522    236858
weighted avg     0.6585    0.7712    0.6877    236858



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:38<00:00,  1.62s/it]

{'n_estimators': 20, 'con': 0.04}
0.4710199738700297
              precision    recall  f1-score   support

           0     0.7871    0.9999    0.8809    185162
           1     0.9879    0.0316    0.0612     51696

    accuracy                         0.7885    236858
   macro avg     0.8875    0.5157    0.4710    236858
weighted avg     0.8310    0.7885    0.7020    236858



: 